In [ ]:
import pupil_classv2 as pc
import math
from statistics import mean
import csv
from timestamps import new_timestamps
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from helper import *
import tdt
from scipy.signal import find_peaks, peak_widths,resample,correlate, correlation_lags,peak_prominences,resample,butter, lfilter, freqz,medfilt
from scipy.stats import pearsonr,wilcoxon,mannwhitneyu,kruskal
import os
from fp_helper import *
from matplotlib import rcParams
import cv2
from scipy.stats import ttest_ind,ttest_rel,zscore
from sklearn.linear_model import Ridge,RidgeCV
from sklearn.model_selection import KFold,cross_val_score
from random import randint
from scipy.ndimage import median_filter
import scikit_posthocs as sp
import random
params = {
        "font.family" : "Arial",
        'pdf.fonttype' : 42,
        'axes.labelsize': 10,
        'axes.titlesize': 11,
        'axes.linewidth': 0.5,
        'xtick.labelsize':9,
        'xtick.major.width':0.5,
        'ytick.major.width':0.5,
        'ytick.labelsize':9,
        'axes.spines.top':False,
        'axes.spines.right':False
         }
rcParams['figure.figsize'] = 21.7,8.27
rcParams.update(params)
def format_ax(ax,xlim,ylim,xspace,yspace):
   

    ax.set_xticks(np.arange(xlim[0],xlim[1]+ xspace,xspace))
    ax.set_yticks(np.arange(ylim[0],ylim[1]+ yspace,yspace))

    ax.set(ylim=(ylim[0], ylim[1]))
    ax.set(xlim=(xlim[0], xlim[1]))


In [ ]:
###single trial single pulse analysis
id_dates = { 
"004104" : ["20230911"],
"004103" : ["20230911"],
"004106" : ["20230912"],
"004107" : ["20230912"],
"004155" : ["20230912"],
"004105" : ["20230911"],
}

df_list = []

sess = 1
pre = 0.5
post = 2
for id,dates in id_dates.items():
    for date in dates:
        pupil_path = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_{sess}_PupilProcessed.csv"
        matpath = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/EtOhBehavior_{id}_{date}_sess_{sess}.mat"
        mat_data = loadmat(matpath)["data"]
        laser_times = mat_data["response"]["laser_time"]
        data = mat_data["response"]["sessdata"]
        laser_durs = mat_data["response"]["trial_dur"]
        face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_{sess}_FaceProcessed.csv"
        diffs = np.array(pd.read_csv(face_path)["Facial Movement"])

        trans = np.transpose(data)
        ard_times = trans[4]

        pupil_df = pd.read_csv(pupil_path)
        times = pupil_df["Time"].to_numpy()

        areas = pupil_df['Pupil Area'].to_numpy()
        wheel = trans[3]
        wheel[0] = 0
        wheel = np.interp(times,ard_times,wheel)
    
        wheel_snips,wheel_peri_times = trial_align(laser_times,times,wheel, pre = pre, post = post)
        pupil_snips,pupil_peri_times = trial_align(laser_times,times,areas,pre = pre, post = post)
        face_snips,face_peri_times = trial_align(laser_times,times,diffs, pre = pre, post = post)


        for j,trial in enumerate(wheel_snips):
            full_baseline = np.nanmean(wheel_snips[j])
            pre_baseline = np.nanmean(wheel_snips[j][0:10])
            during_laser = np.nanmean(wheel_snips[j][40:50])
            for i,time in enumerate(wheel_peri_times):
                new_dict = {"Time from Laser (s)" : time, "trial" : j, "value" : trial[i],"type" : "wheel", "ID" : id, "date" : date,"sess":sess,"pre" : pre_baseline,"during":full_baseline,"during_laser":during_laser}
                df_list.append(new_dict)
        
        for j,trial in enumerate(pupil_snips):
            full_baseline = np.nanmean(wheel_snips[j])
            pre_baseline = np.nanmean(wheel_snips[j][0:10])
            during_laser = np.nanmean(wheel_snips[j][40:50])
            pre_pupil = np.nanmean(trial[0:10])
            trial = trial - pre_pupil
            for i,time in enumerate(pupil_peri_times):
                new_dict = {"Time from Laser (s)" : time, "trial" : j, "value" : trial[i],"type" : "pupil", "ID" : id, "date" : date,"sess":sess,"pre" : pre_baseline,"during":full_baseline,"during_laser":during_laser}
                df_list.append(new_dict)

        
        for j,trial in enumerate(face_snips):
            full_baseline = np.nanmean(wheel_snips[j])
            pre_baseline = np.nanmean(wheel_snips[j][0:10])
            during_laser = np.nanmean(wheel_snips[j][40:50])
            pre_face = np.nanmean(trial[0:10])
            trial = trial - pre_face
            for i,time in enumerate(face_peri_times):
                new_dict = {"Time from Laser (s)" : time, "trial" : j, "value" : trial[i],"type" : "face", "ID" : id, "date" : date,"sess":sess,"pre" : pre_baseline,"during":full_baseline,"during_laser":during_laser}
                df_list.append(new_dict)


trial_df = pd.DataFrame.from_dict(df_list)

In [ ]:
trial_df

In [ ]:
quiet_trial_df = trial_df.query("during<1")
plt.figure(figsize = (1.35,1.5))
ax = sns.lineplot(data = quiet_trial_df.query("type == 'wheel'"),x = "Time from Laser (s)", y = "value",errorbar = "se",linewidth = 1,color = "darkorange",legend = None)
format_ax(ax,(-0.5,2),(-10,70),0.5,10)

ax.axvline(0,linestyle = "dashed", color = "black",linewidth = 0.5)

ax.set_xlabel("Time from Laser (s)")
ax.set_ylabel('Speed (AU)')

#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/single_pulse_quiet_wheel.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
quiet_trial_df["ID"].value_counts()


In [ ]:
quiet_trial_df = trial_df.query("during<1")
plt.figure(figsize = (1.35,1.5))
ax = sns.lineplot(data = quiet_trial_df.query("type == 'pupil'"),x = "Time from Laser (s)", y = "value",errorbar = "se",linewidth = 1,color = "dodgerblue",legend = None)
format_ax(ax,(-0.5,2),(-0.1,0.3),0.5,0.1)

ax.axvline(0,linestyle = "dashed", color = "black",linewidth = 0.5)

ax.set_xlabel("Time from Laser (s)")
ax.set_ylabel("Pupil Size (z-scr)")

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/single_pulse_quiet_pupil.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
quiet_trial_df = trial_df.query("during<1")
plt.figure(figsize = (1.35,1.5))
ax = sns.lineplot(data = quiet_trial_df.query("type == 'face'"),x = "Time from Laser (s)", y = "value",errorbar = "se",linewidth = 1,color = "purple",legend = None)
format_ax(ax,(-0.5,2),(-0.2,0.3),0.5,0.1)

ax.axvline(0,linestyle = "dashed", color = "black",linewidth = 0.5)

ax.set_xlabel("Time from Laser (s)")
ax.set_ylabel("Facial Movement\n(z-scr)")

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/single_pulse_quiet_face.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
quiet_trial_df = trial_df.query("during>5")
plt.figure(figsize = (1.35,1.5))
ax = sns.lineplot(data = quiet_trial_df.query("type == 'wheel'"),x = "Time from Laser (s)", y = "value",errorbar = "se",linewidth = 1,color = "darkorange",legend = None)
format_ax(ax,(-0.5,2),(-10,70),0.5,10)

ax.axvline(0,linestyle = "dashed", color = "black",linewidth = 0.5)

ax.set_xlabel("Time from Laser (s)")
ax.set_ylabel('Speed (AU)')

#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/single_pulse_run_wheel.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
quiet_trial_df["ID"].value_counts()


In [ ]:
quiet_trial_df = trial_df.query("during>5")
plt.figure(figsize = (1.35,1.5))
ax = sns.lineplot(data = quiet_trial_df.query("type == 'pupil'"),x = "Time from Laser (s)", y = "value",errorbar = "se",linewidth = 1,color = "dodgerblue",legend = None)
format_ax(ax,(-0.5,2),(-0.1,0.3),0.5,0.1)

ax.axvline(0,linestyle = "dashed", color = "black",linewidth = 0.5)

ax.set_xlabel("Time from Laser (s)")
ax.set_ylabel("Pupil Size (z-scr)")

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/single_pulse_run_pupil.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
quiet_trial_df = trial_df.query("during>5")
plt.figure(figsize = (1.35,1.5))
ax = sns.lineplot(data = quiet_trial_df.query("type == 'face'"),x = "Time from Laser (s)", y = "value",errorbar = "se",linewidth = 1,color = "purple",legend = None)
format_ax(ax,(-0.5,2),(-0.2,0.3),0.5,0.1)

ax.axvline(0,linestyle = "dashed", color = "black",linewidth = 0.5)

ax.set_xlabel("Time from Laser (s)")
ax.set_ylabel("Facial Movement\n(z-scr)")

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/single_pulse_run_face.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
###single trial single pulse analysis
id_dates = { 
"004104" : ["20230911"],
"004103" : ["20230911"],
"004106" : ["20230912"],
"004107" : ["20230912"],
"004155" : ["20230912"],
"004105" : ["20230911"],
}

df_list = []

sess = 1
pre = 0.5
post = 2
for id,dates in id_dates.items():
    for date in dates:
        pupil_path = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_{sess}_PupilProcessed.csv"
        matpath = rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/EtOhBehavior_{id}_{date}_sess_{sess}.mat"
        mat_data = loadmat(matpath)["data"]
        laser_times = mat_data["response"]["laser_time"]
        data = mat_data["response"]["sessdata"]
        laser_durs = mat_data["response"]["trial_dur"]
        face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_{sess}_FaceProcessed.csv"
        diffs = np.array(pd.read_csv(face_path)["Facial Movement"])

        trans = np.transpose(data)
        ard_times = trans[4]

        pupil_df = pd.read_csv(pupil_path)
        times = pupil_df["Time"].to_numpy()

        areas = pupil_df['Pupil Area'].to_numpy()
        wheel = trans[3]
        wheel[0] = 0
        wheel = np.interp(times,ard_times,wheel)
    
        wheel_snips,wheel_peri_times = trial_align(laser_times,times,wheel, pre = pre, post = post)
        pupil_snips,pupil_peri_times = trial_align(laser_times,times,areas,pre = pre, post = post)
        face_snips,face_peri_times = trial_align(laser_times,times,diffs, pre = pre, post = post)


        for j,trial in enumerate(wheel_snips):
            full_baseline = np.nanmean(wheel_snips[j])
            pre_baseline = np.nanmean(wheel_snips[j][0:10])
            during_laser = np.nanmean(wheel_snips[j][40:50])
            pre_wheel = np.nanmean(trial[0:10])
            post_wheel = np.nanmean(trial[10:50])
            new_dict = { "trial" : j,"type" : "wheel", "ID" : id, "date" : date,"sess":sess,"pre" : pre_baseline,"during":full_baseline,"during_laser":during_laser,"pre-laser":pre_wheel,"post-laser":post_wheel}
            df_list.append(new_dict)
        
        for j,trial in enumerate(pupil_snips):
            full_baseline = np.nanmean(wheel_snips[j])
            pre_baseline = np.nanmean(wheel_snips[j][0:10])
            during_laser = np.nanmean(wheel_snips[j][40:50])
            pre_pupil = np.nanmean(trial[0:10])
            post_pupil = np.nanmean(trial[10:50])
            new_dict = { "trial" : j,"type" : "pupil", "ID" : id, "date" : date,"sess":sess,"pre" : pre_baseline,"during":full_baseline,"during_laser":during_laser,"pre-laser":pre_pupil,"post-laser":post_pupil}
            df_list.append(new_dict)

        
        for j,trial in enumerate(face_snips):
            full_baseline = np.nanmean(wheel_snips[j])
            pre_baseline = np.nanmean(wheel_snips[j][0:10])
            during_laser = np.nanmean(wheel_snips[j][40:50])
            pre_face = np.nanmean(trial[0:10])
            post_face = np.nanmean(trial[10:50])
            new_dict = { "trial" : j,"type" : "face", "ID" : id, "date" : date,"sess":sess,"pre" : pre_baseline,"during":full_baseline,"during_laser":during_laser,"pre-laser":pre_face,"post-laser":post_face}
            df_list.append(new_dict)


bar_df = pd.DataFrame.from_dict(df_list)

In [ ]:
bar_df.query("during>5").query("type == 'wheel'")["ID"].value_counts()

In [ ]:
bar_df.query("during<1").query("type == 'wheel'")["ID"].value_counts()

In [ ]:
###quiet wheel stats

stat_df = bar_df.query("during<1").query("type == 'wheel'")

group1 = stat_df["pre-laser"]
group2 = stat_df["post-laser"]
print(wilcoxon(group1, group2,nan_policy= "omit"))
print(group1.shape,group2.shape)

In [ ]:
###quiet pupil stats

stat_df = bar_df.query("during<1").query("type == 'pupil'")

group1 = stat_df["pre-laser"]
group2 = stat_df["post-laser"]
print(wilcoxon(group1, group2,nan_policy= "omit"))
print(group1.shape,group2.shape)

In [ ]:
###quiet face stats

stat_df = bar_df.query("during<1").query("type == 'face'")

group1 = stat_df["pre-laser"]
group2 = stat_df["post-laser"]
print(wilcoxon(group1, group2,nan_policy= "omit"))
print(group1.shape,group2.shape)

In [ ]:
###run wheel stats

stat_df = bar_df.query("during>5").query("type == 'wheel'")

group1 = stat_df["pre-laser"]
group2 = stat_df["post-laser"]
print(wilcoxon(group1, group2,nan_policy= "omit"))
print(group1.shape,group2.shape)

In [ ]:
###run wheel stats

stat_df = bar_df.query("during>5").query("type == 'pupil'")

group1 = stat_df["pre-laser"]
group2 = stat_df["post-laser"]
print(wilcoxon(group1, group2,nan_policy= "omit"))
print(group1.shape,group2.shape)

In [ ]:
###run wheel stats

stat_df = bar_df.query("during>5").query("type == 'face'")

group1 = stat_df["pre-laser"]
group2 = stat_df["post-laser"]
print(wilcoxon(group1, group2,nan_policy= "omit"))
print(group1.shape,group2.shape)

In [ ]:
###locomotion pupil vs quiet pupil
pupil_df = bar_df.query("type == 'pupil'")
pupil_df["delta"] = pupil_df["post-laser"] - pupil_df["pre-laser"]
quiet_pupil_df = pupil_df.query("during<1")
quiet_pupil_df["condition"] = "quiet"
run_pupil_df = pupil_df.query("during>5")
run_pupil_df["condition"] = "run"
pupil_df = pd.concat([quiet_pupil_df,run_pupil_df])

In [ ]:
###locomotion vs non locomotion stats

group1 = pupil_df.query("condition=='quiet'")
group2 = pupil_df.query("condition=='run'")
mannwhitneyu(group1["delta"], group2["delta"],nan_policy= "omit")